In [1]:
from pathlib import Path

import numpy as np
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

import extract_features

In [29]:
connection_args = {
    'host': 'localhost',
    'dbname': 'audio',
    'port': 5432
}

connection = pg.connect(**connection_args)

In [34]:
cursor.cl

1

In [30]:
cursor = connection.cursor()

In [2]:
cwd = Path.cwd()
seg_npz = cwd / 'data' / 'seg_npz'
segments = sorted(seg for seg in seg_npz.iterdir() if seg.suffix == '.npz')

In [3]:
seg0 = np.load(segments[0])
seg1 = np.load(segments[1])

In [4]:
# last elem is the track title
segs, name = seg0.values()
for idx, seg in enumerate(segs[:1], start=...):
    feats = extract_features.extract_all(seg)

/Users/matteding/miniconda3/envs/audio/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [60]:
idx = 0

In [ ]:
name = np.asscalar(name)

In [67]:
name

'Infinite'

In [31]:
tempo

array([161.49902344])

In [70]:
featscopy = feats.copy()
tempo = featscopy.pop('tempo')
tempo = np.asscalar(tempo)
query_tempo = f"INSERT INTO tempo (song, segment, val) VALUES ('{name}', {idx}, {tempo});"
cursor.execute(query_tempo)

INSERT INTO tempo (song, segment, val) VALUES ('Infinite', 0, 161.4990234375);


In [83]:
cursor.execute('COMMIT;')

In [84]:
cursor.execute('BEGIN;')

In [81]:
cursor.execute("ROLLBACK;")

In [80]:
def map_float_sql(x):
    """Coerce python NaN and +/-Inf values to SQL values"""
    
    if np.isnan(x):
        return "'NaN'"
    if np.isposinf(x):
        return "'+infinity'"
    if np.isneginf(x):
        return "'-infinity'"
    return x

In [82]:
for feat, aggs in list(featscopy.items()):
    dct = aggs._asdict()
    agg_keys = ', '.join(dct.keys())    
    agg_vals = ', '.join(f'{map_float_sql(v)}' for v in dct.values())

    query = f"INSERT INTO {feat} (song, segment, {agg_keys}) VALUES ('{name}', {idx}, {agg_vals});"
    cursor.execute(query)

In [23]:
keys = list(featscopy.keys())
len(keys)

27

In [24]:
keys

['rmse',
 'spectral_bandwidth',
 'spectral_centroid',
 'spectral_flatness',
 'spectral_rolloff',
 'zero_crossing_rate',
 'x_perf5',
 'y_perf5',
 'x_min3',
 'y_min3',
 'x_maj3',
 'y_maj3',
 'bound_width',
 'pitch_0',
 'pitch_1',
 'pitch_2',
 'pitch_3',
 'pitch_4',
 'pitch_5',
 'pitch_6',
 'pitch_7',
 'pitch_8',
 'pitch_9',
 'pitch_10',
 'pitch_11',
 'tempogram',
 'dynamic_tempo']

In [7]:
cursor.description